In [45]:
import agentpy as ap
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import HTML


# ---------------------------------------------------------
# Clase del agente aspiradora
# ---------------------------------------------------------
class Aspiradora(ap.Agent):

    def setup(self):
        # Contador de movimientos que ha hecho el agente
        self.movimientos = 0  

    def step(self):
        # Obtener posición actual
        posicion = self.model.ambiente.positions[self]

        # Si la celda está sucia, se limpia
        if self.model.celdas[posicion] == 1:
            self.model.celdas[posicion] = 0
        else:
            # Elegir una de las 8 direcciones vecinas
            direcciones = [(-1, -1), (-1, 0), (-1, 1),
                           (0, -1),          (0, 1),
                           (1, -1),  (1, 0), (1, 1)]
            d = random.choice(direcciones)
            nueva_pos = (posicion[0] + d[0], posicion[1] + d[1])

            # Verificar que la nueva posición esté dentro de la habitación
            if (0 <= nueva_pos[0] < self.model.alto) and (0 <= nueva_pos[1] < self.model.ancho):
                self.model.ambiente.move_to(self, nueva_pos)
                self.movimientos += 1


# ---------------------------------------------------------
# Clase del modelo (la habitación)
# ---------------------------------------------------------
class Habitacion(ap.Model):

    def setup(self):
        # Parámetros
        self.alto, self.ancho = self.p.tamaño
        self.tiempo_max = self.p.tiempo_max
        self.num_agentes = self.p.num_agentes

        # Ambiente de grilla
        self.ambiente = ap.Grid(self, (self.alto, self.ancho))

        # Inicializar celdas sucias según porcentaje
        total_celdas = self.alto * self.ancho
        suelo_sucio = int(total_celdas * self.p.porc_sucias)
        todas = [(i, j) for i in range(self.alto) for j in range(self.ancho)]
        posiciones_sucias = random.sample(todas, suelo_sucio)
        self.celdas = {pos: 1 if pos in posiciones_sucias else 0 for pos in todas}

        # Crear agentes y ponerlos en la celda (0,0)
        self.agentes = ap.AgentList(self, self.num_agentes, Aspiradora)
        self.ambiente.add_agents(self.agentes, positions=[(0, 0)] * self.num_agentes)

        # Variables de métricas
        self.movimientos_totales = 0
        self.tiempo_limpio = None

    def step(self):
        # Cada agente actúa
        self.agentes.step()
        # Acumular movimientos
        self.movimientos_totales = sum(a.movimientos for a in self.agentes)
        # Verificar si ya no hay celdas sucias
        if sum(self.celdas.values()) == 0 and self.tiempo_limpio is None:
            self.tiempo_limpio = self.t

    def update(self):
        # Detener simulación si ya está limpia la habitación o se alcanzó el tiempo máximo
        if self.tiempo_limpio is not None or self.t >= self.tiempo_max:
            self.stop()

    def end(self):
        # Calcular métricas finales
        total_celdas = self.alto * self.ancho
        limpias = total_celdas - sum(self.celdas.values())
        self.porc_limpias = limpias / total_celdas * 100

        # Guardar resultados en el reporte de AgentPy
        self.report('Tiempo para limpiar', 
                    self.tiempo_limpio if self.tiempo_limpio is not None else 'No completado')
        self.report('Porcentaje final limpio', self.porc_limpias)
        self.report('Movimientos realizados', self.movimientos_totales)


# ---------------------------------------------------------
# Función para graficar cada paso de la simulación
# ---------------------------------------------------------
def mi_grafica(modelo, ax):
    alto, ancho = modelo.alto, modelo.ancho

    # Crear matriz con estado de las celdas (1 = sucia, 0 = limpia)
    matriz = np.zeros((alto, ancho))
    for (i, j), estado in modelo.celdas.items():
        matriz[i, j] = estado

    # Mostrar suelo
    ax.imshow(matriz, cmap="Greys", origin="upper")

    # Dibujar agentes en azul
    for ag in modelo.agentes:
        x, y = modelo.ambiente.positions[ag]
        ax.scatter(y, x, c="blue", s=100, marker="o")

    ax.set_xticks([])
    ax.set_yticks([])


#Datos de entrada para la simulación
parametros = {
    'tamaño': (5, 5),
    'num_agentes': 4,
    'porc_sucias': 0.3,
    'tiempo_max': 60
}




# Animación de la simulación
fig, ax = plt.subplots()
modelo = Habitacion(parametros)
resultados = modelo.run()
print(resultados.reporters)
animacion = ap.animate(modelo, fig, ax, mi_grafica)
HTML(animacion.to_jshtml())





Completed: 26 steps
Run time: 0:00:00.001913
Simulation finished
                                      seed  Tiempo para limpiar  \
0  170303626007451688764578282321509311383                   26   

   Porcentaje final limpio  Movimientos realizados  
0                    100.0                      61  
